<a href="https://colab.research.google.com/github/Blacksujit/Health-Summerize-AI/blob/main/DoctorChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gradio faiss-cpu PyPDF2 openai==0.28 tiktoken -U langchain-community

In [ ]:
import gradio as gr
import openai
import faiss
import numpy as np
import requests
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
import json


In [ ]:
# OpenAI ve diğer API anahtarlarını ayarlayın
openai_api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # OpenAI API anahtarınızı buraya ekleyin
openai.api_key = openai_api_key

weather_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # OpenWeatherMap API anahtarınızı buraya ekleyin
exchange_api_key = "xxxxxxxxxxxxxxxxxxxxxxxxxxxx"  # Exchangeratesapi.io API anahtarınızı buraya ekleyin


In [ ]:
# PDF dosyalarının yolları
pdf_paths = ['/content/Current Essentials of Medicine.pdf'
]

# FAISS indeksi ve belgeler için global değişkenler
vector_index = None
documents = []
chat_history = []

In [ ]:
# PDF'leri okuma ve indeksleme fonksiyonu
def index_pdfs():
    global vector_index, documents

    for pdf_path in pdf_paths:
        pdf_reader = PdfReader(pdf_path)
        text = ""
        for page in pdf_reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text
        documents.append(text)

    combined_text = " ".join(documents)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_text(combined_text)

    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    vectors = embeddings.embed_documents(texts)

    vector_array = np.array(vectors)

    index = faiss.IndexFlatL2(vector_array.shape[1])
    index.add(vector_array)

    vector_index = index

    print("Bilgi tabanı başarıyla oluşturuldu!")

index_pdfs()

<ipython-input-8-b3abdacc56b4>:19: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)


Bilgi tabanı başarıyla oluşturuldu!


In [ ]:
# Hava durumu verilerini çekmek için fonksiyon
def fetch_weather(location):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={weather_api_key}&units=metric&lang=tr"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temp = data['main']['temp']
        weather_description = data['weather'][0]['description']
        return f"{location} için anlık hava durumu: {temp}°C ve {weather_description}."
    else:
        return "Hava durumu bilgilerini alamadım. Lütfen konumu kontrol edip tekrar deneyin."


In [ ]:
# Döviz kuru verilerini çekmek ve döviz dönüşümü yapmak için fonksiyon
def fetch_exchange_rate(base_currency, target_currency, amount=1):
    url = f"https://api.exchangeratesapi.io/v1/latest?access_key={exchange_api_key}&format=1"
    response = requests.get(url)
    data = response.json()
    rates = data.get('rates', {})
    if target_currency in rates:
        rate = rates[target_currency]
        converted_amount = float(amount) * rate
        return f"{amount} {base_currency} = {converted_amount:.2f} {target_currency}."
    else:
        return f"{target_currency} için döviz kuru bilgisi bulunamadı."


In [ ]:
def format_chat_history(chat_history):
    formatted_history = ""
    for entry in chat_history:
        if entry["role"] == "user":
            formatted_history += f"<div class='chat-bubble user'>{entry['content']}</div>"
        else:
            formatted_history += f"<div class='chat-bubble assistant'>{entry['content']}</div>"

    return formatted_history

In [ ]:
# GPT-4 Yanıtını oluşturmak için fonksiyon (function calling ile)
def generate_gpt4_response(prompt_input):
    global vector_index, documents, chat_history
    openai.api_key = openai_api_key

    functions = [
        {
            "name": "fetch_weather",
            "description": "Belirli bir konum için hava durumu bilgisini alır.",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "Hava durumu almak istediğiniz konumun adı"
                    }
                },
                "required": ["location"],
            },
        },
        {
            "name": "fetch_exchange_rate",
            "description": "Belirli iki para birimi arasındaki döviz kuru bilgisini alır ve isteğe bağlı olarak belirli bir miktar için dönüştürme yapar.",
            "parameters": {
                "type": "object",
                "properties": {
                    "base_currency": {
                        "type": "string",
                        "description": "Döviz kurunu almak istediğiniz temel para birimi, varsayılan olarak EUR'dir."
                    },
                    "target_currency": {
                        "type": "string",
                        "description": "Döviz kurunu almak istediğiniz hedef para biriminin ISO 4217 kodu (örneğin: TRY)"
                    },
                    "amount": {
                        "type": "number",
                        "description": "Dönüştürmek istediğiniz miktar (varsayılan olarak 1)."
                    }
                },
                "required": ["base_currency", "target_currency", "amount"],
            },
        }
    ]

    # OpenAI API çağrısı (function calling ile)
    response = openai.ChatCompletion.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system",
             "content": "Sen tıp bilgileri ile donatılmış bir asistansın ve görevin tıbbi konulardaki sorulara cevap vermektir."},
            {"role": "user", "content": prompt_input}
        ],
        functions=functions,
        function_call="auto",  # Modelin fonksiyon çağrısına karar vermesine izin ver
        temperature=0.5,
        max_tokens=512
    )

    # Sohbet geçmişini güncelle
    chat_history.append({"role": "user", "content": prompt_input})

    # Bir fonksiyon çağrısı istenip istenmediğini kontrol et
    if 'choices' in response and response['choices'][0]['finish_reason'] == 'function_call':
        function_call_info = response['choices'][0]['message']['function_call']
        function_name = function_call_info['name']
        arguments = json.loads(function_call_info['arguments'])

        if function_name == 'fetch_weather':
            location = arguments['location']
            weather_response = fetch_weather(location)
            chat_history.append({"role": "assistant", "content": weather_response})
            return format_chat_history(chat_history)
        elif function_name == 'fetch_exchange_rate':
            base_currency = arguments['base_currency']
            target_currency = arguments['target_currency']
            amount = arguments.get('amount', 1)
            exchange_rate_response = fetch_exchange_rate(base_currency, target_currency, amount)
            chat_history.append({"role": "assistant", "content": exchange_rate_response})
            return format_chat_history(chat_history)

    # Asistanın yanıtını al
    assistant_response = response['choices'][0]['message']['content'].strip()
    chat_history.append({"role": "assistant", "content": assistant_response})

    return format_chat_history(chat_history)

In [ ]:
with gr.Blocks() as demo:
    # CSS ile kaydırma özelliği ve stil düzenlemeleri
    custom_css = """
    /* Chat balonları için stil */
    .chat-bubble {
        padding: 10px;
        border-radius: 10px;
        margin-bottom: 10px;
        max-width: 60%;
        word-wrap: break-word;
    }

    .user {
        background-color: #d1e7dd;
        text-align: right;
        margin-left: auto;
    }

    .assistant {
        background-color: #f8d7da;
        text-align: left;
        margin-right: auto;
    }

    /* Kaydırılabilir sohbet kutusu */
    #output-box {
        height: 400px;  /* Sabit yükseklik */
        width: 100%;
        overflow-y: scroll !important;  /* Kaydırmayı gizle ama JS ile açacağız */
        padding: 10px;
        border: 1px solid #ccc;
        border-radius: 10px;
        background-color: #f8f9fa;
        margin-bottom: 20px;
    }

    /* Giriş kutusu */
    #input-box {
        height: 150px;
        width: 100%;
    }
    """

    output_textbox = gr.HTML(label="Yanıt", elem_id="output-box")  # HTML bileşeni ile sohbeti gösteriyoruz
    input_textbox = gr.Textbox(label="Sorunuzu girin", lines=4, elem_id="input-box")  # Giriş alanı

    # Sorgu gönderildiğinde çalıştırılacak fonksiyon
    def on_submit(prompt_input):
        response = generate_gpt4_response(prompt_input)
        return response, ""  # Giriş kutusunu temizle

    # Sohbeti temizlemek için fonksiyon
    def clear_chat():
        global chat_history
        chat_history.clear()
        return "", ""  # Hem giriş kutusunu hem de sohbeti temizle

    # Giriş kutusuna "Enter" basıldığında çalıştırılan fonksiyon
    input_textbox.submit(on_submit, inputs=input_textbox, outputs=[output_textbox, input_textbox])

    # Mesaj gönderme ve sohbeti temizleme butonları
    submit_btn = gr.Button("Gönder")
    clear_btn = gr.Button("Chat'i Temizle")

    submit_btn.click(on_submit, inputs=input_textbox, outputs=[output_textbox, input_textbox])
    clear_btn.click(clear_chat, outputs=[output_textbox, input_textbox])

    demo.css = custom_css
    demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1046f76164fb165e08.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://b162ad70725dccd76c.gradio.live
Killing tunnel 127.0.0.1:7861 <> https://2eae138fb007aae19d.gradio.live
Killing tunnel 127.0.0.1:7862 <> https://1dcffbd5ed4db3ce8d.gradio.live
Killing tunnel 127.0.0.1:7863 <> https://c9b054571b98902445.gradio.live
Killing tunnel 127.0.0.1:7864 <> https://6311d0f05878886aba.gradio.live
Killing tunnel 127.0.0.1:7865 <> https://44bc8391d8b84b0e1d.gradio.live
Killing tunnel 127.0.0.1:7866 <> https://1046f76164fb165e08.gradio.live


In [1]:
!pip install gradio transformers torch pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import gradio as gr
from transformers import BlipProcessor, BlipForConditionalGeneration, AutoModelForCausalLM, AutoTokenizer
from PIL import Image, ImageOps
import torch
import logging
from datetime import datetime
import re
import os

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    filename='medical_analysis.log'
)
logger = logging.getLogger(__name__)

# Ensure offline mode for transformers
os.environ["TRANSFORMERS_OFFLINE"] = "1"

class MedicalImageAnalyzer:
    def __init__(self):
        """Initialize the medical image analysis system."""
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.image_processor = None
        self.image_model = None
        self.nlp_model = None
        self.nlp_tokenizer = None
        self.load_models()

    def load_models(self):
        """Load all required models locally."""
        try:
            logger.info("Loading BLIP model for image analysis...")
            self.image_processor = BlipProcessor.from_pretrained(
                "Salesforce/blip-image-captioning-base",
                cache_dir="./model_cache"
            )
            self.image_model = BlipForConditionalGeneration.from_pretrained(
                "Salesforce/blip-image-captioning-base",
                cache_dir="./model_cache"
            ).to(self.device)

            logger.info("Loading local NLP model...")
            self.nlp_tokenizer = AutoTokenizer.from_pretrained(
                "stanford-crfm/BioMedLM",
                cache_dir="./model_cache"
            )
            self.nlp_model = AutoModelForCausalLM.from_pretrained(
                "stanford-crfm/BioMedLM",
                cache_dir="./model_cache"
            ).to(self.device)

            logger.info("All models loaded successfully.")
        except Exception as e:
            logger.error(f"Model loading failed: {str(e)}")
            raise RuntimeError("Failed to initialize models. Ensure models are available locally.")

    def analyze_image(self, image: Image.Image, clinical_context: str = "") -> dict:
        """Analyze medical image with clinical context."""
        try:
            if not image:
                return {"error": "No image provided"}

            # Preprocess image
            processed_image = self._preprocess_image(image)

            # Generate findings
            findings = self._generate_findings(processed_image, clinical_context)

            # Generate recommendations
            recommendations = self._generate_recommendations(findings)

            # Structure the final report
            report = self._structure_report(clinical_context, findings, recommendations)

            return report
        except Exception as e:
            logger.error(f"Analysis failed: {str(e)}")
            return {"error": str(e)}

    def _preprocess_image(self, image: Image.Image) -> Image.Image:
        """Preprocess medical image for analysis."""
        try:
            if image.mode != 'RGB':
                image = image.convert('RGB')
            image = ImageOps.exif_transpose(image)
            return image
        except Exception as e:
            logger.error(f"Image preprocessing failed: {str(e)}")
            raise RuntimeError("Image preprocessing failed.")

    def _generate_findings(self, image: Image.Image, context: str) -> str:
        """Generate findings from the image."""
        try:
            prompt = (
                f"Analyze the medical image and provide a detailed report. "
                f"Clinical context: {context if context else 'None provided'}. "
                "Include details about anatomical structures, abnormalities, and technical quality."
            )
            inputs = self.image_processor(image, text=prompt, return_tensors="pt").to(self.device)

            with torch.no_grad():
                outputs = self.image_model.generate(**inputs, max_new_tokens=200)

            findings = self.image_processor.decode(outputs[0], skip_special_tokens=True)
            return self._clean_text(findings)
        except Exception as e:
            logger.error(f"Findings generation failed: {str(e)}")
            raise RuntimeError("Could not generate findings.")

    def _generate_recommendations(self, findings: str) -> str:
        """Generate clinical recommendations."""
        try:
            prompt = (
                f"Based on the following findings:\n{findings}\n\n"
                "Provide 3-5 clinical recommendations. Categorize them as urgent or routine."
            )
            inputs = self.nlp_tokenizer(prompt, return_tensors="pt").to(self.device)

            with torch.no_grad():
                outputs = self.nlp_model.generate(**inputs, max_new_tokens=150, temperature=0.7)

            recommendations = self.nlp_tokenizer.decode(outputs[0], skip_special_tokens=True)
            return self._clean_text(recommendations)
        except Exception as e:
            logger.error(f"Recommendation generation failed: {str(e)}")
            return "Could not generate recommendations."

    def _structure_report(self, context: str, findings: str, recommendations: str) -> dict:
        """Structure the final report."""
        return {
            "metadata": {
                "report_date": datetime.now().isoformat(),
                "analysis_version": "1.0"
            },
            "clinical_context": context,
            "findings": findings,
            "recommendations": recommendations
        }

    def _clean_text(self, text: str) -> str:
        """Clean up the generated text."""
        text = text.strip()
        text = re.sub(r"\s+", " ", text)  # Remove extra whitespace
        return text

# Initialize the analyzer
try:
    analyzer = MedicalImageAnalyzer()
except Exception as e:
    logger.critical(f"System initialization failed: {str(e)}")
    raise RuntimeError("Medical analysis system failed to initialize.")

def analyze_medical_image(image, clinical_context=""):
    """Wrapper function for Gradio interface."""
    try:
        if not image:
            return "⚠️ Please upload a medical image."

        # Perform analysis
        report = analyzer.analyze_image(image, clinical_context)

        if "error" in report:
            return f"❌ Error: {report['error']}"

        # Format the report
        return (
            f"**Clinical Context**: {clinical_context if clinical_context else 'None provided'}\n\n"
            f"**Findings**:\n{report['findings']}\n\n"
            f"**Recommendations**:\n{report['recommendations']}\n\n"
            "Note: This analysis requires verification by a qualified radiologist."
        )
    except Exception as e:
        return f"❌ System Error: {str(e)}"

# Gradio Interface
with gr.Blocks(theme=gr.themes.Soft(), title="Medical Image Analysis") as app:
    gr.Markdown("# 🩺 Medical Image Analysis")
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Medical Image")
            context_input = gr.Textbox(label="Clinical Context (optional)", placeholder="Patient symptoms or history...")
            analyze_btn = gr.Button("Analyze", variant="primary")
        with gr.Column():
            report_output = gr.Markdown(label="Analysis Report")

    analyze_btn.click(
        analyze_medical_image,
        inputs=[image_input, context_input],
        outputs=[report_output]
    )

if __name__ == "__main__":
    app.launch(server_name="0.0.0.0", server_port=78)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ed690cf29681d2f068.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
# Install required libraries
!pip install gradio transformers torch pillow datasets

# Import necessary libraries
import os
import logging
import re
from datetime import datetime
from PIL import Image, ImageOps
import torch
from transformers import (
    BlipProcessor,
    BlipForConditionalGeneration,
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset
import gradio as gr

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"

# Step 1: Fine-Tune the BLIP Model for Image Analysis
def fine_tune_blip():
    logger.info("Loading spinal cord dataset...")
    dataset_path = "/content/spinal_cord_dataset"  # Kaggle dataset path
    dataset = load_dataset("imagefolder", data_dir=dataset_path, split="train[:1%]")  # Use a small subset for demonstration

    logger.info("Loading BLIP model and processor...")
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

    def preprocess_function(examples):
        inputs = processor(text=examples["label"], images=examples["image"], return_tensors="pt", padding=True)
        return inputs

    logger.info("Preprocessing dataset...")
    tokenized_dataset = dataset.map(preprocess_function, batched=True)

    logger.info("Setting up training arguments...")
    training_args = TrainingArguments(
        output_dir="./blip_results",
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=4,
        num_train_epochs=1,  # Set to 1 for demonstration; increase for real training
        save_steps=10_000,
        save_total_limit=2,
        remove_unused_columns=False,
    )

    logger.info("Starting fine-tuning...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
    )
    trainer.train()

    logger.info("Fine-tuning complete. Saving model...")
    model.save_pretrained("./fine_tuned_blip")
    processor.save_pretrained("./fine_tuned_blip")

# Step 2: Fine-Tune the GPT-2 Model for Text Generation
def fine_tune_gpt2():
    logger.info("Loading radiology reports dataset...")
    dataset_path = "/content/spinal_cord_dataset"  # Kaggle dataset path
    dataset = load_dataset("text", data_files={"train": os.path.join(dataset_path, "reports.txt")}, split="train")

    logger.info("Loading GPT-2 model and tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained("gpt2")
    model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

    def preprocess_function(examples):
        return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

    logger.info("Preprocessing dataset...")
    tokenized_dataset = dataset.map(preprocess_function, batched=True)

    logger.info("Setting up training arguments...")
    training_args = TrainingArguments(
        output_dir="./gpt2_results",
        evaluation_strategy="epoch",
        learning_rate=5e-5,
        per_device_train_batch_size=4,
        num_train_epochs=1,  # Set to 1 for demonstration; increase for real training
        save_steps=10_000,
        save_total_limit=2,
    )

    logger.info("Starting fine-tuning...")
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_dataset,
    )
    trainer.train()

    logger.info("Fine-tuning complete. Saving model...")
    model.save_pretrained("./fine_tuned_gpt2")
    tokenizer.save_pretrained("./fine_tuned_gpt2")

# Step 3: Load Fine-Tuned Models for Inference
class MedicalImageAnalyzer:
    def __init__(self):
        logger.info("Loading fine-tuned BLIP model...")
        self.image_processor = BlipProcessor.from_pretrained("./fine_tuned_blip")
        self.image_model = BlipForConditionalGeneration.from_pretrained("./fine_tuned_blip").to(device)

        logger.info("Loading fine-tuned GPT-2 model...")
        self.nlp_tokenizer = AutoTokenizer.from_pretrained("./fine_tuned_gpt2")
        self.nlp_model = AutoModelForCausalLM.from_pretrained("./fine_tuned_gpt2").to(device)

    def analyze_image(self, image: Image.Image, clinical_context: str = "") -> dict:
        try:
            if not image:
                return {"error": "No image provided"}

            # Preprocess image
            processed_image = self._preprocess_image(image)

            # Generate findings
            findings = self._generate_findings(processed_image, clinical_context)

            # Generate recommendations
            recommendations = self._generate_recommendations(findings)

            # Structure the final report
            report = self._structure_report(clinical_context, findings, recommendations)

            return report
        except Exception as e:
            logger.error(f"Analysis failed: {str(e)}")
            return {"error": str(e)}

    def _preprocess_image(self, image: Image.Image) -> Image.Image:
        if image.mode != "RGB":
            image = image.convert("RGB")
        return ImageOps.exif_transpose(image)

    def _generate_findings(self, image: Image.Image, context: str) -> str:
        prompt = (
            f"Analyze the medical image and provide a detailed report. "
            f"Clinical context: {context if context else 'None provided'}. "
            "Include details about anatomical structures, abnormalities, and technical quality."
        )
        inputs = self.image_processor(image, text=prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = self.image_model.generate(**inputs, max_new_tokens=200)

        findings = self.image_processor.decode(outputs[0], skip_special_tokens=True)
        return findings.strip()

    def _generate_recommendations(self, findings: str) -> str:
        prompt = (
            f"Based on the following findings:\n{findings}\n\n"
            "Provide 3-5 clinical recommendations. Categorize them as urgent or routine."
        )
        inputs = self.nlp_tokenizer(prompt, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = self.nlp_model.generate(**inputs, max_new_tokens=150, temperature=0.7)

        recommendations = self.nlp_tokenizer.decode(outputs[0], skip_special_tokens=True)
        return recommendations.strip()

    def _structure_report(self, context: str, findings: str, recommendations: str) -> dict:
        return {
            "metadata": {
                "report_date": datetime.now().isoformat(),
                "analysis_version": "1.0",
            },
            "clinical_context": context,
            "findings": findings,
            "recommendations": recommendations,
        }

# Step 4: Deploy the Model on Gradio
def analyze_medical_image(image, clinical_context=""):
    analyzer = MedicalImageAnalyzer()
    report = analyzer.analyze_image(image, clinical_context)

    if "error" in report:
        return f"❌ Error: {report['error']}"

    return (
        f"**Clinical Context**: {clinical_context if clinical_context else 'None provided'}\n\n"
        f"**Findings**:\n{report['findings']}\n\n"
        f"**Recommendations**:\n{report['recommendations']}\n\n"
        "Note: This analysis requires verification by a qualified radiologist."
    )

with gr.Blocks() as app:
    gr.Markdown("# 🩺 Medical Image Analysis")
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload Medical Image")
            context_input = gr.Textbox(label="Clinical Context (optional)", placeholder="Patient symptoms or history...")
            analyze_btn = gr.Button("Analyze", variant="primary")
        with gr.Column():
            report_output = gr.Markdown(label="Analysis Report")

    analyze_btn.click(
        analyze_medical_image,
        inputs=[image_input, context_input],
        outputs=[report_output],
    )

app.launch(server_name="0.0.0.0", server_port=6)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [2]:
# spinal_cord_dataset_path = !kagglehub.dataset_download('trainingdatapro/spinal-cord-dataset')

In [1]:
!pip install kagglehub
# Install Kaggle API
!pip install kaggle

In [2]:
# Upload kaggle.json
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json



cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [3]:
# Download the dataset
!kaggle datasets download -d trainingdatapro/spinal-cord-dataset


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 10, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/cli.py", line 68, in main
    out = args.func(**command_args)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 1741, in dataset_download_cli
    with self.build_kaggle_client() as kaggle:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/kaggle/api/kaggle_api_extended.py", line 688, in build_kaggle_client
    username=self.config_values['username'],
             ~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^
KeyError: 'username'


In [5]:
# import os
# from kaggle.api.kaggle_api_extended import KaggleApi

# # Set up Kaggle API credentials
# os.environ["KAGGLE_USERNAME"] = "sujitnirmal"
# os.environ["KAGGLE_KEY"] = "591a53c307288d0b46fc0a7e25d60531"

# # Initialize the Kaggle API
# api = KaggleApi()
# api.authenticate()

# # Download the spinal-cord-dataset
# dataset_name = "trainingdatapro/spinal-cord-dataset"
# download_path = "./spinal_cord_dataset"
# api.dataset_download_files(dataset_name, path=download_path, unzip=True)

# print(f"Dataset downloaded and extracted to: {download_path}")

In [ ]:
# Install required libraries
!pip install gradio transformers torch pillow datasets

# Import necessary libraries
import os
import logging
import re
from datetime import datetime
from PIL import Image, ImageOps
import torch
from transformers import (
    BlipProcessor,
    BlipForConditionalGeneration,
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset
import gradio as gr

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")

# Define paths
BLIP_MODEL_PATH = "./fine_tuned_blip"
GPT2_MODEL_PATH = "./fine_tuned_gpt2"
DATASET_PATH = "/content/spinal_cord_dataset/ST000001"  # Update this to your actual dataset path

# Step 1: Fine-Tune the BLIP Model for Image Analysis
def fine_tune_blip(dataset_path=DATASET_PATH, force_retrain=False):
    """Fine-tune BLIP model if not already fine-tuned or if force_retrain is True."""
    if os.path.exists(BLIP_MODEL_PATH) and not force_retrain:
        logger.info(f"BLIP model already fine-tuned at {BLIP_MODEL_PATH}. Skipping training.")
        return

    # Ensure dataset directory exists
    if not os.path.exists(dataset_path):
        logger.error(f"Dataset not found at {dataset_path}. Please download the dataset first.")
        raise FileNotFoundError(f"Dataset not found at {dataset_path}")

    try:
        logger.info("Loading spinal cord dataset...")
        # Try to load as image folder dataset
        try:
            dataset = load_dataset("imagefolder", data_dir=dataset_path, split="train[:1%]")
        except Exception as e:
            logger.warning(f"Failed to load dataset as imagefolder: {e}")
            # Fallback to using a different method or path structure
            dataset = load_dataset(dataset_path, split="train[:1%]")

        logger.info("Loading BLIP model and processor...")
        processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

        def preprocess_function(examples):
            # Ensure 'label' exists in the dataset, otherwise use empty strings
            labels = examples.get("label", [""] * len(examples["image"]))
            inputs = processor(images=examples["image"], text=labels, return_tensors="pt", padding=True)
            return inputs

        logger.info("Preprocessing dataset...")
        tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)

        logger.info("Setting up training arguments...")
        training_args = TrainingArguments(
            output_dir="./blip_results",
            evaluation_strategy="epoch",
            learning_rate=5e-5,
            per_device_train_batch_size=4,
            num_train_epochs=1,  # Set to 1 for demonstration; increase for real training
            save_steps=10_000,
            save_total_limit=2,
            remove_unused_columns=False,
        )

        logger.info("Starting fine-tuning...")
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset,
        )
        trainer.train()

        logger.info("Fine-tuning complete. Saving model...")
        model.save_pretrained(BLIP_MODEL_PATH)
        processor.save_pretrained(BLIP_MODEL_PATH)
        return True
    except Exception as e:
        logger.error(f"Failed to fine-tune BLIP model: {e}")
        raise

# Step 2: Fine-Tune the GPT-2 Model for Text Generation
def fine_tune_gpt2(dataset_path=DATASET_PATH, force_retrain=False):
    """Fine-tune GPT-2 model if not already fine-tuned or if force_retrain is True."""
    if os.path.exists(GPT2_MODEL_PATH) and not force_retrain:
        logger.info(f"GPT-2 model already fine-tuned at {GPT2_MODEL_PATH}. Skipping training.")
        return

    # Ensure dataset directory exists
    if not os.path.exists(dataset_path):
        logger.error(f"Dataset not found at {dataset_path}. Please download the dataset first.")
        raise FileNotFoundError(f"Dataset not found at {dataset_path}")

    reports_file = os.path.join(dataset_path, "reports.txt")
    if not os.path.exists(reports_file):
        logger.error(f"Reports file not found at {reports_file}")
        raise FileNotFoundError(f"Reports file not found at {reports_file}")

    try:
        logger.info("Loading radiology reports dataset...")
        dataset = load_dataset("text", data_files={"train": reports_file}, split="train")

        logger.info("Loading GPT-2 model and tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained("gpt2")
        model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

        def preprocess_function(examples):
            return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

        logger.info("Preprocessing dataset...")
        tokenized_dataset = dataset.map(preprocess_function, batched=True)

        logger.info("Setting up training arguments...")
        training_args = TrainingArguments(
            output_dir="./gpt2_results",
            evaluation_strategy="epoch",
            learning_rate=5e-5,
            per_device_train_batch_size=4,
            num_train_epochs=1,  # Set to 1 for demonstration; increase for real training
            save_steps=10_000,
            save_total_limit=2,
        )

        logger.info("Starting fine-tuning...")
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset,
        )
        trainer.train()

        logger.info("Fine-tuning complete. Saving model...")
        model.save_pretrained(GPT2_MODEL_PATH)
        tokenizer.save_pretrained(GPT2_MODEL_PATH)
        return True
    except Exception as e:
        logger.error(f"Failed to fine-tune GPT-2 model: {e}")
        raise

# Step 3: Create a Medical Image Analyzer class that can use either fine-tuned or pretrained models
class MedicalImageAnalyzer:
    def __init__(self, use_fine_tuned=True):
        """
        Initialize the analyzer with either fine-tuned or pretrained models.

        Args:
            use_fine_tuned: If True, try to load fine-tuned models. If they don't exist,
                            fall back to pretrained models.
        """
        self.use_fine_tuned = use_fine_tuned

        # Load BLIP model
        logger.info("Loading BLIP model...")
        if use_fine_tuned and os.path.exists(BLIP_MODEL_PATH):
            try:
                self.image_processor = BlipProcessor.from_pretrained(BLIP_MODEL_PATH)
                self.image_model = BlipForConditionalGeneration.from_pretrained(BLIP_MODEL_PATH).to(device)
                logger.info("Successfully loaded fine-tuned BLIP model")
            except Exception as e:
                logger.warning(f"Error loading fine-tuned BLIP model: {e}. Falling back to pretrained model.")
                self.image_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
                self.image_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
        else:
            logger.info("Using pretrained BLIP model")
            self.image_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
            self.image_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

        # Load GPT-2 model
        logger.info("Loading GPT-2 model...")
        if use_fine_tuned and os.path.exists(GPT2_MODEL_PATH):
            try:
                self.nlp_tokenizer = AutoTokenizer.from_pretrained(GPT2_MODEL_PATH)
                self.nlp_model = AutoModelForCausalLM.from_pretrained(GPT2_MODEL_PATH).to(device)
                logger.info("Successfully loaded fine-tuned GPT-2 model")
            except Exception as e:
                logger.warning(f"Error loading fine-tuned GPT-2 model: {e}. Falling back to pretrained model.")
                self.nlp_tokenizer = AutoTokenizer.from_pretrained("gpt2")
                self.nlp_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
        else:
            logger.info("Using pretrained GPT-2 model")
            self.nlp_tokenizer = AutoTokenizer.from_pretrained("gpt2")
            self.nlp_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

    def analyze_image(self, image: Image.Image, clinical_context: str = "") -> dict:
        """
        Analyze a medical image with optional clinical context.

        Args:
            image: PIL Image object
            clinical_context: Optional string with clinical information

        Returns:
            dict: Analysis report with findings and recommendations
        """
        try:
            if image is None:
                return {"error": "No image provided"}

            # Preprocess image
            processed_image = self._preprocess_image(image)

            # Generate findings
            findings = self._generate_findings(processed_image, clinical_context)

            # Generate recommendations
            recommendations = self._generate_recommendations(findings)

            # Structure the final report
            report = self._structure_report(clinical_context, findings, recommendations)

            return report
        except Exception as e:
            logger.error(f"Analysis failed: {str(e)}")
            return {"error": str(e)}

    def _preprocess_image(self, image: Image.Image) -> Image.Image:
        """Preprocess the image for model input."""
        # Handle different image modes
        if image.mode != "RGB":
            image = image.convert("RGB")
        # Fix orientation based on EXIF data
        return ImageOps.exif_transpose(image)

    def _generate_findings(self, image: Image.Image, context: str) -> str:
        """Generate findings from the image using the BLIP model."""
        prompt = (
            f"Analyze the medical image and provide a detailed report. "
            f"Clinical context: {context if context else 'None provided'}. "
            "Include details about anatomical structures, abnormalities, and technical quality."
        )
        # Process inputs
        inputs = self.image_processor(image, text=prompt, return_tensors="pt").to(device)

        # Generate output
        with torch.no_grad():
            outputs = self.image_model.generate(**inputs, max_new_tokens=200)

        # Decode output
        findings = self.image_processor.decode(outputs[0], skip_special_tokens=True)
        return findings.strip()

    def _generate_recommendations(self, findings: str) -> str:
        """Generate recommendations based on findings using the GPT-2 model."""
        prompt = (
            f"Based on the following findings:\n{findings}\n\n"
            "Provide 3-5 clinical recommendations. Categorize them as urgent or routine."
        )
        # Process inputs
        inputs = self.nlp_tokenizer(prompt, return_tensors="pt").to(device)

        # Generate output
        with torch.no_grad():
            outputs = self.nlp_model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                top_p=0.95
            )

        # Decode output
        recommendations = self.nlp_tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the generated recommendations (remove the prompt)
        recommendations = recommendations[len(prompt):].strip()
        return recommendations

    def _structure_report(self, context: str, findings: str, recommendations: str) -> dict:
        """Structure the analysis results into a report."""
        return {
            "metadata": {
                "report_date": datetime.now().isoformat(),
                "analysis_version": "1.0",
                "model_type": "fine-tuned" if self.use_fine_tuned else "pretrained",
            },
            "clinical_context": context,
            "findings": findings,
            "recommendations": recommendations,
        }

# Step 4: Create functions for Gradio interface
def initialize_analyzer():
    """Initialize the analyzer once and return it for reuse."""
    # Check if fine-tuned models exist, otherwise use pretrained models
    use_fine_tuned = os.path.exists(BLIP_MODEL_PATH) and os.path.exists(GPT2_MODEL_PATH)
    return MedicalImageAnalyzer(use_fine_tuned=use_fine_tuned)

# Global variable to store the analyzer once initialized
analyzer = None

def analyze_medical_image(image, clinical_context=""):
    """Function to analyze medical image for Gradio interface."""
    global analyzer

    try:
        # Initialize analyzer if not already done
        if analyzer is None:
            analyzer = initialize_analyzer()

        if image is None:
            return "❌ Error: No image provided. Please upload an image first."

        # Analyze image
        report = analyzer.analyze_image(image, clinical_context)

        if "error" in report:
            return f"❌ Error: {report['error']}"

        # Format the report for display
        return (
            f"**Clinical Context**: {clinical_context if clinical_context else 'None provided'}\n\n"
            f"**Findings**:\n{report['findings']}\n\n"
            f"**Recommendations**:\n{report['recommendations']}\n\n"
            f"**Note**: This analysis was generated using "
            f"{'fine-tuned' if analyzer.use_fine_tuned else 'pretrained'} models.\n"
            f"This analysis requires verification by a qualified radiologist."
        )
    except Exception as e:
        logger.error(f"Error during analysis: {str(e)}")
        return f"❌ Error during analysis: {str(e)}"

def get_model_status():
    """Get the status of the models (fine-tuned or pretrained)."""
    blip_status = "Fine-tuned" if os.path.exists(BLIP_MODEL_PATH) else "Pretrained"
    gpt2_status = "Fine-tuned" if os.path.exists(GPT2_MODEL_PATH) else "Pretrained"
    return f"BLIP Model: {blip_status}\nGPT-2 Model: {gpt2_status}"

def start_fine_tuning():
    """Start the fine-tuning process."""
    try:
        fine_tune_blip()
        fine_tune_gpt2()
        # Reinitialize analyzer with newly fine-tuned models
        global analyzer
        analyzer = initialize_analyzer()
        return get_model_status() + "\n\n✅ Fine-tuning completed successfully! Models are ready to use."
    except Exception as e:
        return f"❌ Fine-tuning failed: {str(e)}"

# Step 5: Create Gradio interface
def create_gradio_app():
    """Create and launch the Gradio application."""
    with gr.Blocks() as app:
        gr.Markdown("# 🩺 Medical Image Analysis")

        with gr.Row():
            with gr.Column():
                # Model status display
                status_output = gr.Markdown(value=get_model_status())
                fine_tune_btn = gr.Button("Fine-tune Models", variant="secondary")

                # User inputs
                image_input = gr.Image(type="pil", label="Upload Medical Image")
                context_input = gr.Textbox(
                    label="Clinical Context (optional)",
                    placeholder="Patient symptoms or history...",
                    lines=3
                )
                analyze_btn = gr.Button("Analyze Image", variant="primary")

            with gr.Column():
                # Analysis output
                report_output = gr.Markdown(label="Analysis Report")

        # Set up event handlers
        fine_tune_btn.click(
            start_fine_tuning,
            inputs=[],
            outputs=[status_output]
        )

        analyze_btn.click(
            analyze_medical_image,
            inputs=[image_input, context_input],
            outputs=[report_output]
        )

        # In newer Gradio versions, we don't use update() method
        # Instead, we set the initial value when creating the component

    return app

# Main execution
if __name__ == "__main__":
    # Create and launch app
    app = create_gradio_app()
    app.launch(server_name="0.0.0.0", server_port=200, share=True)

In [17]:

 # Install required libraries
!pip install gradio transformers torch pillow datasets kaggle

# Install Kaggle for dataset download (if needed)
try:
    import kaggle
    print("Kaggle already installed")
except ImportError:
    print("Installing Kaggle...")
    !pip install kaggle

# Import necessary libraries
import os
import logging
import re
from datetime import datetime
from PIL import Image, ImageOps
import torch
from transformers import (
    BlipProcessor,
    BlipForConditionalGeneration,
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
)
from datasets import load_dataset
import gradio as gr

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
)
logger = logging.getLogger(__name__)

# Set device
device = "cuda" if torch.cuda.is_available() else "cpu"
logger.info(f"Using device: {device}")

# Define paths
BLIP_MODEL_PATH = "./fine_tuned_blip"
GPT2_MODEL_PATH = "./fine_tuned_gpt2"
DATASET_PATH = "/content/spinal_cord_dataset"  # Local path where dataset will be downloaded/stored

# Function to download the dataset from Kaggle if not already present
def download_dataset(force_download=False):
    """
    Download the spinal cord dataset from Kaggle if not already downloaded.

    Args:
        force_download: If True, redownload even if files exist

    Returns:
        str: Path to the downloaded dataset
    """
    if os.path.exists(DATASET_PATH) and not force_download:
        logger.info(f"Dataset already exists at {DATASET_PATH}")
        return DATASET_PATH

    try:
        # Check if Kaggle credentials exist
        import kaggle
        logger.info("Attempting to download dataset from Kaggle...")

        # Create dataset directory if it doesn't exist
        os.makedirs(DATASET_PATH, exist_ok=True)

        # Download the dataset using the Kaggle API
        # Note: This requires a kaggle.json file in ~/.kaggle/ with API credentials
        try:
            # Try to download directly using the kaggle API
            kaggle.api.authenticate()
            kaggle.api.dataset_download_files(
                "trainingdatapro/spinal-cord-dataset",
                path=DATASET_PATH,
                unzip=True
            )
            logger.info(f"Successfully downloaded dataset to {DATASET_PATH}")
            return DATASET_PATH
        except Exception as e:
            logger.warning(f"Failed to download using API: {e}")

            # Alternative method using shell commands
            logger.info("Trying alternative download method...")
            try:
                !mkdir -p {DATASET_PATH}
                !kaggle datasets download -d trainingdatapro/spinal-cord-dataset -p {DATASET_PATH}
                !unzip {DATASET_PATH}/spinal-cord-dataset.zip -d {DATASET_PATH}
                logger.info(f"Successfully downloaded dataset to {DATASET_PATH}")
                return DATASET_PATH
            except Exception as e2:
                logger.error(f"Failed to download dataset: {e2}")
                raise
    except ImportError:
        logger.error("Kaggle not installed or configured properly")
        logger.info("Please install Kaggle and set up credentials manually:")
        logger.info("1. pip install kaggle")
        logger.info("2. Create ~/.kaggle/kaggle.json with your API key from kaggle.com/account")
        logger.info("3. chmod 600 ~/.kaggle/kaggle.json")
        logger.info(f"4. Download dataset from kaggle.com/trainingdatapro/spinal-cord-dataset and extract to {DATASET_PATH}")
        raise

    return DATASET_PATH

# Step 1: Fine-Tune the BLIP Model for Image Analysis
def fine_tune_blip(dataset_path=DATASET_PATH, force_retrain=False):
    """Fine-tune BLIP model if not already fine-tuned or if force_retrain is True."""
    if os.path.exists(BLIP_MODEL_PATH) and not force_retrain:
        logger.info(f"BLIP model already fine-tuned at {BLIP_MODEL_PATH}. Skipping training.")
        return

    # Download/check dataset
    try:
        dataset_path = download_dataset()
    except Exception as e:
        logger.error(f"Failed to download dataset: {e}")
        raise

    # Ensure dataset directory exists
    if not os.path.exists(dataset_path):
        logger.error(f"Dataset not found at {dataset_path}. Please download the dataset first.")
        raise FileNotFoundError(f"Dataset not found at {dataset_path}")

    try:
        logger.info("Loading spinal cord dataset...")

        # Try to determine the dataset structure
        # First, check if there's an images folder
        images_folder = os.path.join(dataset_path, "images")
        if os.path.exists(images_folder):
            logger.info(f"Found images folder at {images_folder}")
            dataset_path = images_folder

        # Check if there are image files directly in the dataset folder
        image_files = [f for f in os.listdir(dataset_path) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
        if image_files:
            logger.info(f"Found {len(image_files)} image files in {dataset_path}")
        else:
            logger.warning(f"No image files found in {dataset_path}")
            # Look for subdirectories that might contain images
            for root, dirs, files in os.walk(dataset_path):
                images = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]
                if images:
                    logger.info(f"Found {len(images)} images in {root}")
                    dataset_path = root
                    break

        # Try to load as image folder dataset
        try:
            dataset = load_dataset("imagefolder", data_dir=dataset_path, split="train[:1%]")
            logger.info(f"Successfully loaded dataset as imagefolder from {dataset_path}")
        except Exception as e:
            logger.warning(f"Failed to load dataset as imagefolder: {e}")
            # Try to load from a text file with image paths
            try:
                # Look for a metadata file or labels file
                metadata_files = [f for f in os.listdir(dataset_path) if f.lower().endswith(('.csv', '.txt'))]
                if metadata_files:
                    logger.info(f"Found metadata files: {metadata_files}")
                    # For now, just use the images directly, without labels

                # Fallback to creating a simple dataset from image files
                image_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path)
                              if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff'))]

                if not image_files:
                    raise ValueError(f"No image files found in {dataset_path}")

                logger.info(f"Creating dataset from {len(image_files)} image files")

                # Create a simple dataset with image paths
                from datasets import Dataset
                dataset = Dataset.from_dict({
                    "image": image_files,
                    "label": [""] * len(image_files)  # Empty labels for now
                })

                # Load images
                def load_image(example):
                    example["image"] = Image.open(example["image"]).convert("RGB")
                    return example

                dataset = dataset.map(load_image)
                logger.info("Successfully created dataset from image files")
            except Exception as e2:
                logger.error(f"Failed to create dataset: {e2}")
                raise

        logger.info("Loading BLIP model and processor...")
        processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

        def preprocess_function(examples):
            # Ensure 'label' exists in the dataset, otherwise use empty strings
            labels = examples.get("label", [""] * len(examples["image"]))
            # Process images and text
            try:
                inputs = processor(images=examples["image"], text=labels, return_tensors="pt", padding=True)
                return inputs
            except Exception as e:
                logger.error(f"Error processing examples: {e}")
                # Print example structure for debugging
                logger.info(f"Example keys: {examples.keys()}")
                logger.info(f"Image type: {type(examples['image'][0])}")
                raise

        logger.info("Preprocessing dataset...")
        # Get dataset column names before mapping
        column_names = dataset.column_names
        logger.info(f"Dataset column names: {column_names}")

        tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=column_names)

        logger.info("Setting up training arguments...")
        training_args = TrainingArguments(
            output_dir="./blip_results",
            evaluation_strategy="epoch",
            learning_rate=5e-5,
            per_device_train_batch_size=4,
            num_train_epochs=1,  # Set to 1 for demonstration; increase for real training
            save_steps=10_000,
            save_total_limit=2,
            remove_unused_columns=False,
        )

        logger.info("Starting fine-tuning...")
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset,
        )
        trainer.train()

        logger.info("Fine-tuning complete. Saving model...")
        model.save_pretrained(BLIP_MODEL_PATH)
        processor.save_pretrained(BLIP_MODEL_PATH)
        return True
    except Exception as e:
        logger.error(f"Failed to fine-tune BLIP model: {e}")
        raise

# Step 2: Fine-Tune the GPT-2 Model for Text Generation
def fine_tune_gpt2(dataset_path=DATASET_PATH, force_retrain=False):
    """Fine-tune GPT-2 model if not already fine-tuned or if force_retrain is True."""
    if os.path.exists(GPT2_MODEL_PATH) and not force_retrain:
        logger.info(f"GPT-2 model already fine-tuned at {GPT2_MODEL_PATH}. Skipping training.")
        return

    # Download/check dataset
    try:
        dataset_path = download_dataset()
    except Exception as e:
        logger.error(f"Failed to download dataset: {e}")
        raise

    # Ensure dataset directory exists
    if not os.path.exists(dataset_path):
        logger.error(f"Dataset not found at {dataset_path}. Please download the dataset first.")
        raise FileNotFoundError(f"Dataset not found at {dataset_path}")

    try:
        logger.info("Looking for text data for radiology reports...")

        # Try to find text files containing reports
        reports_file = None
        reports_content = []

        # Search for potential report files
        for root, dirs, files in os.walk(dataset_path):
            for filename in files:
                # Look for text files that might contain reports
                if filename.lower().endswith('.txt') and 'report' in filename.lower():
                    reports_file = os.path.join(root, filename)
                    logger.info(f"Found potential reports file: {reports_file}")
                    break
                # Also consider CSV files that might contain report data
                elif filename.lower().endswith('.csv') and 'report' in filename.lower():
                    reports_file = os.path.join(root, filename)
                    logger.info(f"Found potential reports CSV: {reports_file}")
                    break

            if reports_file:
                break

        # If no specific reports file is found, try to extract text from any text files
        if not reports_file:
            logger.info("No specific reports file found, searching for any text files...")
            text_files = []
            for root, dirs, files in os.walk(dataset_path):
                for filename in files:
                    if filename.lower().endswith('.txt'):
                        text_files.append(os.path.join(root, filename))

            if text_files:
                logger.info(f"Found {len(text_files)} general text files")
                reports_file = text_files[0]  # Use the first text file
            else:
                logger.warning("No text files found in the dataset")

        # If we found a reports file, read its content
        if reports_file:
            logger.info(f"Reading reports from {reports_file}")
            try:
                with open(reports_file, 'r', encoding='utf-8') as f:
                    content = f.read()
                    reports_content = [content]
            except Exception as e:
                logger.error(f"Error reading reports file: {e}")
                raise

        # If no reports file or content, create synthetic data for demonstration
        if not reports_content:
            logger.warning("No reports content found. Creating synthetic data for demonstration purposes.")
            reports_content = [
                "FINDINGS: MRI of the thoracic spine demonstrates normal alignment. The vertebral body heights are preserved. "
                "There is no evidence of compression fracture or significant degenerative changes. "
                "The spinal cord demonstrates normal signal intensity throughout the thoracic region. "
                "No evidence of cord compression, edema, or syrinx formation. "
                "The intervertebral discs demonstrate normal signal intensity and height. "
                "No evidence of significant disc bulge or herniation. "
                "The paravertebral soft tissues are normal in appearance.",

                "FINDINGS: MRI of the lumbar spine reveals mild degenerative disc disease at L4-L5 and L5-S1 levels. "
                "There is a small central disc protrusion at L4-L5 without significant neural compression. "
                "No evidence of spinal stenosis. The vertebral body heights are maintained. "
                "The conus medullaris terminates at an appropriate level and appears normal in signal intensity. "
                "The paraspinal soft tissues are unremarkable.",

                "FINDINGS: Cervical spine MRI demonstrates straightening of the normal cervical lordosis, "
                "suggestive of muscle spasm. Mild disc desiccation is noted at C5-C6 and C6-C7 levels. "
                "There is a small right paracentral disc protrusion at C5-C6 causing mild right foraminal narrowing. "
                "No significant central canal stenosis. The spinal cord demonstrates normal signal intensity. "
                "No evidence of cord compression or myelomalacia."
            ]

        # Create a dataset from the reports
        from datasets import Dataset
        dataset = Dataset.from_dict({"text": reports_content})

        logger.info("Loading GPT-2 model and tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained("gpt2")
        tokenizer.pad_token = tokenizer.eos_token  # Set pad token for GPT-2
        model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

        def preprocess_function(examples):
            return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

        logger.info("Preprocessing dataset...")
        tokenized_dataset = dataset.map(preprocess_function, batched=True)

        logger.info("Setting up training arguments...")
        training_args = TrainingArguments(
            output_dir="./gpt2_results",
            evaluation_strategy="epoch",
            learning_rate=5e-5,
            per_device_train_batch_size=4,
            num_train_epochs=1,  # Set to 1 for demonstration; increase for real training
            save_steps=10_000,
            save_total_limit=2,
        )

        logger.info("Starting fine-tuning...")
        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=tokenized_dataset,
        )
        trainer.train()

        logger.info("Fine-tuning complete. Saving model...")
        model.save_pretrained(GPT2_MODEL_PATH)
        tokenizer.save_pretrained(GPT2_MODEL_PATH)
        return True
    except Exception as e:
        logger.error(f"Failed to fine-tune GPT-2 model: {e}")
        raise

# Step 3: Create a Medical Image Analyzer class that can use either fine-tuned or pretrained models
class MedicalImageAnalyzer:
    def __init__(self, use_fine_tuned=True):
        """
        Initialize the analyzer with either fine-tuned or pretrained models.

        Args:
            use_fine_tuned: If True, try to load fine-tuned models. If they don't exist,
                            fall back to pretrained models.
        """
        self.use_fine_tuned = use_fine_tuned

        # Load BLIP model
        logger.info("Loading BLIP model...")
        if use_fine_tuned and os.path.exists(BLIP_MODEL_PATH):
            try:
                self.image_processor = BlipProcessor.from_pretrained(BLIP_MODEL_PATH)
                self.image_model = BlipForConditionalGeneration.from_pretrained(BLIP_MODEL_PATH).to(device)
                logger.info("Successfully loaded fine-tuned BLIP model")
            except Exception as e:
                logger.warning(f"Error loading fine-tuned BLIP model: {e}. Falling back to pretrained model.")
                self.image_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
                self.image_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)
        else:
            logger.info("Using pretrained BLIP model")
            self.image_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
            self.image_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

        # Load GPT-2 model
        logger.info("Loading GPT-2 model...")
        if use_fine_tuned and os.path.exists(GPT2_MODEL_PATH):
            try:
                self.nlp_tokenizer = AutoTokenizer.from_pretrained(GPT2_MODEL_PATH)
                self.nlp_model = AutoModelForCausalLM.from_pretrained(GPT2_MODEL_PATH).to(device)
                logger.info("Successfully loaded fine-tuned GPT-2 model")
            except Exception as e:
                logger.warning(f"Error loading fine-tuned GPT-2 model: {e}. Falling back to pretrained model.")
                self.nlp_tokenizer = AutoTokenizer.from_pretrained("gpt2")
                self.nlp_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)
        else:
            logger.info("Using pretrained GPT-2 model")
            self.nlp_tokenizer = AutoTokenizer.from_pretrained("gpt2")
            self.nlp_model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)

    def analyze_image(self, image: Image.Image, clinical_context: str = "") -> dict:
        """
        Analyze a medical image with optional clinical context.

        Args:
            image: PIL Image object
            clinical_context: Optional string with clinical information

        Returns:
            dict: Analysis report with findings and recommendations
        """
        try:
            if image is None:
                return {"error": "No image provided"}

            # Preprocess image
            processed_image = self._preprocess_image(image)

            # Generate findings
            findings = self._generate_findings(processed_image, clinical_context)

            # Generate recommendations
            recommendations = self._generate_recommendations(findings)

            # Structure the final report
            report = self._structure_report(clinical_context, findings, recommendations)

            return report
        except Exception as e:
            logger.error(f"Analysis failed: {str(e)}")
            return {"error": str(e)}

    def _preprocess_image(self, image: Image.Image) -> Image.Image:
        """Preprocess the image for model input."""
        # Handle different image modes
        if image.mode != "RGB":
            image = image.convert("RGB")
        # Fix orientation based on EXIF data
        return ImageOps.exif_transpose(image)

    def _generate_findings(self, image: Image.Image, context: str) -> str:
        """Generate findings from the image using the BLIP model."""
        prompt = (
            f"Analyze the medical image and provide a detailed report. "
            f"Clinical context: {context if context else 'None provided'}. "
            "Include details about anatomical structures, abnormalities, and technical quality."
        )
        # Process inputs
        inputs = self.image_processor(image, text=prompt, return_tensors="pt").to(device)

        # Generate output
        with torch.no_grad():
            outputs = self.image_model.generate(**inputs, max_new_tokens=200)

        # Decode output
        findings = self.image_processor.decode(outputs[0], skip_special_tokens=True)
        return findings.strip()

    def _generate_recommendations(self, findings: str) -> str:
        """Generate recommendations based on findings using the GPT-2 model."""
        prompt = (
            f"Based on the following findings:\n{findings}\n\n"
            "Provide 3-5 clinical recommendations. Categorize them as urgent or routine."
        )
        # Process inputs
        inputs = self.nlp_tokenizer(prompt, return_tensors="pt").to(device)

        # Generate output
        with torch.no_grad():
            outputs = self.nlp_model.generate(
                **inputs,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                top_p=0.95
            )

        # Decode output
        recommendations = self.nlp_tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract only the generated recommendations (remove the prompt)
        recommendations = recommendations[len(prompt):].strip()
        return recommendations

    def _structure_report(self, context: str, findings: str, recommendations: str) -> dict:
        """Structure the analysis results into a report."""
        return {
            "metadata": {
                "report_date": datetime.now().isoformat(),
                "analysis_version": "1.0",
                "model_type": "fine-tuned" if self.use_fine_tuned else "pretrained",
            },
            "clinical_context": context,
            "findings": findings,
            "recommendations": recommendations,
        }

# Step 4: Create functions for Gradio interface
def initialize_analyzer():
    """Initialize the analyzer once and return it for reuse."""
    # Check if fine-tuned models exist, otherwise use pretrained models
    use_fine_tuned = os.path.exists(BLIP_MODEL_PATH) and os.path.exists(GPT2_MODEL_PATH)
    return MedicalImageAnalyzer(use_fine_tuned=use_fine_tuned)

# Global variable to store the analyzer once initialized
analyzer = None

def analyze_medical_image(image, clinical_context=""):
    """Function to analyze medical image for Gradio interface."""
    global analyzer

    try:
        # Initialize analyzer if not already done
        if analyzer is None:
            analyzer = initialize_analyzer()

        if image is None:
            return "❌ Error: No image provided. Please upload an image first."

        # Analyze image
        report = analyzer.analyze_image(image, clinical_context)

        if "error" in report:
            return f"❌ Error: {report['error']}"

        # Format the report for display
        return (
            f"**Clinical Context**: {clinical_context if clinical_context else 'None provided'}\n\n"
            f"**Findings**:\n{report['findings']}\n\n"
            f"**Recommendations**:\n{report['recommendations']}\n\n"
            f"**Note**: This analysis was generated using "
            f"{'fine-tuned' if analyzer.use_fine_tuned else 'pretrained'} models.\n"
            f"This analysis requires verification by a qualified radiologist."
        )
    except Exception as e:
        logger.error(f"Error during analysis: {str(e)}")
        return f"❌ Error during analysis: {str(e)}"

def get_model_status():
    """Get the status of the models (fine-tuned or pretrained)."""
    blip_status = "Fine-tuned" if os.path.exists(BLIP_MODEL_PATH) else "Pretrained"
    gpt2_status = "Fine-tuned" if os.path.exists(GPT2_MODEL_PATH) else "Pretrained"
    return f"BLIP Model: {blip_status}\nGPT-2 Model: {gpt2_status}"

def start_fine_tuning():
    """Start the fine-tuning process."""
    try:
        fine_tune_blip()
        fine_tune_gpt2()
        # Reinitialize analyzer with newly fine-tuned models
        global analyzer
        analyzer = initialize_analyzer()
        return get_model_status() + "\n\n✅ Fine-tuning completed successfully! Models are ready to use."
    except Exception as e:
        return f"❌ Fine-tuning failed: {str(e)}"

# Step 5: Create Gradio interface
def create_gradio_app():
    """Create and launch the Gradio application."""
    with gr.Blocks() as app:
        gr.Markdown("# 🩺 Medical Image Analysis")

        with gr.Row():
            with gr.Column():
                # Model status display
                status_output = gr.Markdown(value=get_model_status())
                fine_tune_btn = gr.Button("Fine-tune Models", variant="secondary")

                # User inputs
                image_input = gr.Image(type="pil", label="Upload Medical Image")
                context_input = gr.Textbox(
                    label="Clinical Context (optional)",
                    placeholder="Patient symptoms or history...",
                    lines=3
                )
                analyze_btn = gr.Button("Analyze Image", variant="primary")

            with gr.Column():
                # Analysis output
                report_output = gr.Markdown(label="Analysis Report")

        # Set up event handlers
        fine_tune_btn.click(
            start_fine_tuning,
            inputs=[],
            outputs=[status_output]
        )

        analyze_btn.click(
            analyze_medical_image,
            inputs=[image_input, context_input],
            outputs=[report_output]
        )

        # In newer Gradio versions, we don't use update() method
        # Instead, we set the initial value when creating the component

    return app

# Main execution
if __name__ == "__main__":
    # Create and launch app
    app = create_gradio_app()
    app.launch(server_name="0.0.0.0", server_port=12, share=True)

Kaggle already installed
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d9435ec922b7b8ccbe.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
import os
from kaggle.api.kaggle_api_extended import KaggleApi

# Set up Kaggle API credentials
os.environ["KAGGLE_USERNAME"] = "sujitnirmal"
os.environ["KAGGLE_KEY"] = "591a53c307288d0b46fc0a7e25d60531"

# Initialize the Kaggle API
api = KaggleApi()
api.authenticate()

# Download the spinal-cord-dataset
dataset_name = "charan3341/mimic-cxrt"
download_path = "./spinal_cord_dataset"
api.dataset_download_files(dataset_name, path=download_path, unzip=True)

print(f"Dataset downloaded and extracted to: {download_path}")

Dataset URL: https://www.kaggle.com/datasets/charan3341/mimic-cxrt


HTTPError: 403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/datasets/download/charan3341/mimic-cxrt?raw=false

In [ ]:
# mimic_cxr_path = kagglehub.dataset_download('charan3341/mimic-cxr')